# Indus Valley Scripts - ICIT coded Text Analysis for Decipherment

Dataset was created as a csv file from ICIT web site from raw html files for each for the Text
Data labels were changes and a linearized copy of the original text was added

icit_text_corpus.csv


!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install nltk
!pip install ipywidgets
!pip install -U dill
!pip3 install requests

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import collections


plt.style.use(style='seaborn')
%matplotlib inline

In [2]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [3]:
orig_df=pd.read_csv('../../IndusCorpusUtils/data/icit_corpus/icit_text_corpus.csv',dtype=str, nrows=100)
# set the max columns to none
pd.set_option('display.max_columns', None)

orig_df

,icit_id,site,area,section,block,house,cisi,m77_id,excavation,id_no,reference,period,phase,boss,shape,depth,cross_section,artefact_type,color,horizontal,vertical,thickness,horizontal_mm,vertical_mm,thickness_min_mm,thickness_max_mm,material,museum,text_condition,artefact_preservation,cult,symbol,notes,inscribed_sides,style_group,text_class,lines,direction,text,signs,complete,alignment,sign_height,text_images,linearized_text,standardized_text,keywords,symbol.1,symbol_subtype,animal_facing,cult_object,complete.1,iconography_images
0,1,Alamgirpur,NaN,NaN,NaN,NaN,NaN,9061,NaN,NaN,IAR:1958-59:Pl.LXV.A,NaN,NaN,NaN,NaN,NaN,NaN,POT:T:g,NaN,NaN,NaN,NaN,0,0,0,0,Clay,NaN,Fair,complete,None,None,NaN,1,NaN,SS,1,L/R,+410-017+,2,Y,Unordered,Unequal,signs/sign410.jpg signs/sign017.jpg,410 017,410 017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Alamgirpur,NaN,NaN,NaN,NaN,NaN,9062,NaN,NaN,IAR:1958-59:Pl.LXV.A,NaN,NaN,NaN,NaN,NaN,NaN,POT:T:g,NaN,NaN,NaN,NaN,0,0,0,0,Clay,NaN,Fair,complete,None,None,NaN,1,NaN,SS,1,L/R,+410-017+,2,Y,NaN,NaN,signs/sign410.jpg signs/sign017.jpg,410 017,410 017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Alamgirpur,NaN,NaN,NaN,NaN,NaN,9063,NaN,NaN,IAR:1958-59:Pl.LXV.A,NaN,NaN,NaN,NaN,NaN,NaN,POT:T:g,NaN,NaN,NaN,NaN,0,0,0,0,Clay,NaN,Fair,complete,None,None,NaN,1,NaN,SC,1,L/R,+405-017+,2,Y,NaN,NaN,signs/sign405.jpg signs/sign017.jpg,405 017,405 017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Allahdino,NaN,NaN,J6,E1/2,<,0,AD73-157,NaN,NaN,NaN,NaN,NaN,NaN,-125,NaN,POT:T:g,Red,2.5cm,3.0cm,0.6cm,25,30,6,0,Clay,NaN,Poor,fragment,None,None,NaN,1,NaN,??,1,NaN,+220-000+,1,N,NaN,NaN,signs/sign220.jpg signs/sign000.jpg,220 000,000 220,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Allahdino,NaN,NaN,H5,E1/2,<,0,AD73-157,NaN,NaN,NaN,NaN,NaN,NaN,-0.35,NaN,TAG,Red,3.7cm,4.4cm,1.2cm,37,44,12,0,Clay,NaN,Poor,chipped,NaN,Bull,NaN,1,NaN,UC,1,R/L,+740-235+,2,Y,NaN,NaN,signs/sign740.jpg signs/sign235.jpg,740 235,235 740,Bull,Bull,Bull,R,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Chanhujo-daro,NaN,NaN,NaN,NaN,NaN,6227,CH 2468,Pl. LII:27,ASI 63.12.1,NaN,NaN,PB,square,+9.8,NaN,SEAL:S,White,1.1,1.09,0.55,27.9,27.7,0,14,Steatite,NaN,Fine,complete,SAN,Bull1:J,NaN,1,MD-CD#1,SP,1,R/L,+740-803-233-741-250-579+,6,Y,Linear,Adapted,signs/sign740.jpg signs/sign803.jpg signs/sig...,740 803 233 741 250 579,579 250 741 233 803 740,Bull1,Bull1,Bull1:J:c,R,SAN,Y,NaN
96,97,Chanhujo-daro,NaN,NaN,NaN,NaN,NaN,6127,CH 3045,Pl. LI:27,ASI 63.12.21,NaN,NaN,PB,square,+7.6,NaN,SEAL:S,White,1.1,1.09,0.61,27.9,27.7,0,15.5,Steatite,NaN,Good,complete,SAN,Bull1:J,NaN,1,NaN,SC,1,R/L,+151-233-803-033-705-240+,6,Y,Partly linear,Adapted,signs/sign151.jpg signs/sign233.jpg signs/sig...,151 233 803 033 705 240,240 705 033 803 233 151,Bull1,Bull1,Bull1:J:c,R,SAN,Y,NaN
97,98,Chanhujo-daro,NaN,NaN,NaN,NaN,NaN,6129,CH 2297,Pl. LI:29,ASI 63.12.5,NaN,NaN,PB,square,+10,NaN,SEAL:S,White,1.1,1.1,0.41,27.9,27.9,0,10.4,Steatite,NaN,Poor,complete,SAN,Bull1:W,Upper left corner is missing,1,NaN,SP,1,R/L,+740-231-220-032-002-817+,6,Y,Linear,Adapted,signs/sign740.jpg signs/sign231.jpg signs/sig...,740 231 220 032 002 817,817 002 032 220 231 740,Bull1,Bull1,Bull1:W:c,R,SAN,Y,NaN
98,99,Chanhujo-daro,NaN,NaN,NaN,NaN,NaN,6220,CH 2559,Pl. LII:20,ASI 63.12.2,NaN,NaN,PB,square,+9.5,NaN,SEAL:S,White,1,1,0.52,25.4,25.4,0,13.2,Steatite,NaN,Poor,complete,NaN,Bull1:I,NaN,1,NaN,2L,2,R/L,+590-615-140/232-240-001-060-920+,8,Y,Linear,Unequal,signs/sign590.jpg signs/sign615.jpg signs/sig...,590 615 140 232 240 001 060 920,920 060 001 240 232 140 615 590,Bull1,Bull1,Bull1:I:c,R,NaN,N,NaN


In [4]:
#Reverse text and add that as a new column
list_reversed_text = []
for text in orig_df[orig_df.standardized_text!=''].standardized_text:
    # Tokenize to words
    # first split the string into chars
    chars = text.split(' ')

    # then reverse the split string list and join with a space
    reversed_text = ' '.join(reversed(chars))
    list_reversed_text.append(reversed_text)
    
orig_df['reversed_text']= list_reversed_text
    
#remove the values where the text is unclear
df = orig_df[orig_df['standardized_text'].str.contains('000') == False] 

#keep only the values where the direction is unknown (if direction does not have a /, i,e. L/R or R/L)
df = df[df['direction'].str.contains('/') == True] 

#Keep the items with unclear text in another dataframe
df_unclear = orig_df[orig_df['standardized_text'].str.contains('000') == True]

#Note: Lot of the text with unclear text have direction empty

In [5]:
y=df['site'].values
y.shape

(69,)

In [6]:
x=df['standardized_text'].values
x.shape

x_rev=df['reversed_text'].values
x_rev.shape

(69,)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
(x_train,x_test,y_train,y_test)=train_test_split(x,y,test_size=0.1, random_state=43)

(x_rev_train,x_rev_test,y_train,y_test)=train_test_split(x_rev,y,test_size=0.1, random_state=43)

In [9]:
#Train data
df_train_x=pd.DataFrame(x_train)
df_train_x=df_train_x.rename(columns={0:'standardized_text'})

df_train_x_rev=pd.DataFrame(x_rev_train)
df_train_x_rev=df_train_x_rev.rename(columns={0:'reversed_text'})

#Test data
df_test_x=pd.DataFrame(x_test)
df_test_x=df_test_x.rename(columns={0:'standardized_text'})

df_test_x_rev=pd.DataFrame(x_rev_test)
df_test_x_rev=df_test_x_rev.rename(columns={0:'reversed_text'})

In [10]:
print(df_train_x)
print(df_train_x_rev)

                      standardized_text
0                           705 050 740
1                   817 002 803 233 520
2                   235 705 033 904 740
3                   817 002 803 235 520
4                               014 220
..                                  ...
57  034 287 365 002 101 740 140 840 031
58                              817 002
59                      240 100 740 151
60              579 250 741 233 803 740
61          920 060 740 031 590 390 161

[62 rows x 1 columns]
                          reversed_text
0                           740 050 705
1                   520 233 803 002 817
2                   740 904 033 705 235
3                   520 235 803 002 817
4                               220 014
..                                  ...
57  031 840 140 740 101 002 365 287 034
58                              002 817
59                      151 740 100 240
60              740 803 233 741 250 579
61          161 390 590 031 740 060 920

[62 rows x 1 col

In [11]:
print(df_test_x)
print(df_test_x_rev)

             standardized_text
0          861 002 032 176 740
1      817 002 032 220 231 740
2                      803 740
3                      061 740
4  413 125 350 297 002 003 226
5              033 125 390 368
6                 055[ 740 790
                 reversed_text
0          740 176 032 002 861
1      740 231 220 032 002 817
2                      740 803
3                      740 061
4  226 003 002 297 350 125 413
5              368 390 125 033
6                 790 740 055[


In [12]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize

In [13]:
#Tokenize the text

#paddedLine = [list(pad_both_ends(word_tokenize(s), n=2))] -> TBD: Need to pad (https://stackoverflow.com/questions/54959340/nltk-language-modeling-confusion)
#train, vocab = padded_everygram_pipeline(2, paddedLine)
#ut_ngrams = ( ngram for sent in gutenberg.sents() for ngram in ngrams(sent, 3, pad_left = True, pad_right = True, right_pad_symbol='EOS', left_pad_symbol="BOS"))

# See https://stackoverflow.com/questions/54989825/nltk-perplexity-measure-inversion also for Perplexity

#tokenized_text = [list(pad_both_ends(df_train_x[df_train_x.standardized_text!=''].standardized_text.apply(word_tokenize), n=2))]
#reverse_tokenized_text = [list(pad_both_ends(df_train_x_rev[df_train_x_rev.reversed_text!=''].reversed_text.apply(word_tokenize), n=2))]
#print("tokenized_text:",tokenized_text)

tokenized_text = list(df_train_x[df_train_x.standardized_text!=''].standardized_text.apply(word_tokenize))
reverse_tokenized_text = list(df_train_x_rev[df_train_x_rev.reversed_text!=''].reversed_text.apply(word_tokenize))

#print("tokenized_text:",tokenized_text)

In [14]:
# Preprocess the tokenized text for n-grams language modeling
n = 3
import array as arr
train_data_list_fwd_bigram = [None,None, None, None, None,None, None]
padded_sents_list_fwd_bigram = [None,None, None, None, None,None, None]
train_data_list_rev_bigram = [None,None, None, None, None,None, None]
padded_sents_list_rev_bigram = [None,None, None, None, None,None, None]

train_data_list_fwd_trigram = [None,None, None, None, None,None, None]
padded_sents_list_fwd_trigram = [None,None, None, None, None,None, None]
train_data_list_rev_trigram = [None,None, None, None, None,None, None]
padded_sents_list_rev_trigram = [None,None, None, None, None,None, None]

train_data_rev_list = [None,None, None, None, None,None, None]
padded_sents_rev_list = [None,None, None, None, None,None, None]

train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
train_data_rev, padded_sents_rev = padded_everygram_pipeline(n, reverse_tokenized_text)

for index in range (0,7):
    train_data_list_fwd_bigram[index], padded_sents_list_fwd_bigram[index] = padded_everygram_pipeline(2, tokenized_text)
    train_data_list_rev_bigram[index], padded_sents_list_rev_bigram[index] = padded_everygram_pipeline(2, reverse_tokenized_text)
    
    train_data_list_fwd_trigram[index], padded_sents_list_fwd_trigram[index] = padded_everygram_pipeline(3, tokenized_text)
    train_data_list_rev_trigram[index], padded_sents_list_rev_trigram[index] = padded_everygram_pipeline(3, reverse_tokenized_text)

for ngramlize_sent in train_data_list_fwd_trigram[0]:
    print(list(ngramlize_sent))
    print()
print('#############')
list(padded_sents_list_fwd_trigram[0])
    

[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', '705'), ('<s>',), ('<s>', '705'), ('<s>', '705', '050'), ('705',), ('705', '050'), ('705', '050', '740'), ('050',), ('050', '740'), ('050', '740', '</s>'), ('740',), ('740', '</s>'), ('740', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]

[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', '817'), ('<s>',), ('<s>', '817'), ('<s>', '817', '002'), ('817',), ('817', '002'), ('817', '002', '803'), ('002',), ('002', '803'), ('002', '803', '233'), ('803',), ('803', '233'), ('803', '233', '520'), ('233',), ('233', '520'), ('233', '520', '</s>'), ('520',), ('520', '</s>'), ('520', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]

[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', '235'), ('<s>',), ('<s>', '235'), ('<s>', '235', '705'), ('235',), ('235', '705'), ('235', '705', '033'), ('705',), ('705', '033'), ('705', '033', '904'), ('033',), ('033', '904'), ('033', '904', '740'), ('904',), ('904', '740'), ('904', '740', '</s>'), ('740',), ('740', '</s>'

['<s>',
 '<s>',
 '705',
 '050',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '817',
 '002',
 '803',
 '233',
 '520',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '235',
 '705',
 '033',
 '904',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '817',
 '002',
 '803',
 '235',
 '520',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '014',
 '220',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '317',
 '920',
 '002',
 '016',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '456',
 '741',
 '740',
 '530',
 '032',
 '850',
 '031',
 '032',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '797',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '634',
 '368',
 '002',
 '061',
 '717',
 '390',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '892',
 '647',
 '741',
 '067',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '817',
 '002',
 '240',
 '636',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '692',
 '060',
 '531',
 '740',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '717',
 '575',
 '060',
 '844',
 '390',
 '</s>',
 '</s>',
 '<s>',
 '<s>',
 '235',
 '240',
 '740',
 '151',
 '031',
 '032',
 '</s>',
 '</s

In [15]:
# Train Bigram, Trigram Models for both fwd text and reverse tex with the following
# models. Ignoring AbsoluteDiscountingInterpolated model
from nltk.lm.models import MLE
from nltk.lm.models import AbsoluteDiscountingInterpolated
from nltk.lm.models import KneserNeyInterpolated
from nltk.lm.models import Laplace
from nltk.lm.models import Lidstone
from nltk.lm.models import StupidBackoff
from nltk.lm.models import WittenBellInterpolated

gamma=0.5
order=1
model_name_list = ["MLE","KneserNeyInterpolated", "Laplace", "Lidstone","StupidBackoff", "WittenBellInterpolated"]

model_MLE_list_fwd = []
model_KneserNeyInterpolated_list_fwd = []
model_Laplace_list_fwd = []
model_Lidstone_list_fwd = []
model_StupidBackoff_list_fwd = []
model_WittenBellInterpolated_list_fwd= []

model_MLE_list_rev = []
model_KneserNeyInterpolated_list_rev = []
model_Laplace_list_rev = []
model_Lidstone_list_rev = []
model_StupidBackoff_list_rev = []
model_WittenBellInterpolated_list_rev= []

for index in range(2, 4):
    model_MLE_list_fwd.append(MLE(index))
    model_KneserNeyInterpolated_list_fwd.append(KneserNeyInterpolated(index))
    model_Laplace_list_fwd.append(Laplace(index))
    model_Lidstone_list_fwd.append(Lidstone(index, gamma))
    model_StupidBackoff_list_fwd.append(StupidBackoff(index, order))
    model_WittenBellInterpolated_list_fwd.append(WittenBellInterpolated(index))
    
for index in range(2, 4):
    model_MLE_list_rev.append(MLE(index))
    model_KneserNeyInterpolated_list_rev.append(KneserNeyInterpolated(index))
    model_Laplace_list_rev.append(Laplace(index))
    model_Lidstone_list_rev.append(Lidstone(index, gamma))
    model_StupidBackoff_list_rev.append(StupidBackoff(index, order))
    model_WittenBellInterpolated_list_rev.append(WittenBellInterpolated(index))

models_list_fwd_bigram = [model_MLE_list_fwd[0] ,model_KneserNeyInterpolated_list_fwd[0] ,model_Laplace_list_fwd[0] , model_Lidstone_list_fwd[0] , model_StupidBackoff_list_fwd[0] , model_WittenBellInterpolated_list_fwd[0]]
models_list_rev_bigram = [model_MLE_list_rev[0] ,model_KneserNeyInterpolated_list_rev[0] ,model_Laplace_list_rev[0] , model_Lidstone_list_rev[0] , model_StupidBackoff_list_rev[0] , model_WittenBellInterpolated_list_rev[0]]

models_list_fwd_trigram = [model_MLE_list_fwd[1] ,model_KneserNeyInterpolated_list_fwd[1] ,model_Laplace_list_fwd[1] , model_Lidstone_list_fwd[1] , model_StupidBackoff_list_fwd[1] , model_WittenBellInterpolated_list_fwd[1]]
models_list_rev_trigram = [model_MLE_list_rev[1] ,model_KneserNeyInterpolated_list_rev[1] ,model_Laplace_list_rev[1] , model_Lidstone_list_rev[1] , model_StupidBackoff_list_rev[1] , model_WittenBellInterpolated_list_rev[1]]


In [16]:
def fit_and_train_models(name, models_list, train_data_list,padded_sents_list):
    
    for index in range (0,len(models_list)-1):
        models_list[index].fit(train_data_list[index], padded_sents_list[index])
        print("Fit & Train:", name, model_name_list[index], models_list[index].vocab)

In [17]:
fit_and_train_models("Fwd Bigram Model:", models_list_fwd_bigram , train_data_list_fwd_bigram,padded_sents_list_fwd_bigram)
fit_and_train_models("Rev Bigram Model:", models_list_rev_bigram , train_data_list_rev_bigram,padded_sents_list_rev_bigram)

fit_and_train_models("Fwd Trigram Model:", models_list_fwd_trigram , train_data_list_fwd_trigram,padded_sents_list_fwd_trigram)
fit_and_train_models("Rev Trigram Model:", models_list_rev_trigram , train_data_list_rev_trigram,padded_sents_list_rev_trigram)
    

Fit & Train: Fwd Bigram Model: MLE <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Fwd Bigram Model: KneserNeyInterpolated <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Fwd Bigram Model: Laplace <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Fwd Bigram Model: Lidstone <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Fwd Bigram Model: StupidBackoff <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Rev Bigram Model: MLE <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Rev Bigram Model: KneserNeyInterpolated <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Rev Bigram Model: Laplace <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Rev Bigram Model: Lidstone <Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
Fit & Train: Rev Bigram Model: StupidBackoff <Vocabulary with cutoff=1 unk_label='<UNK>' a

In [18]:
#Check one of the models and play with it
model = models_list_fwd_trigram[1] #KneserNeyInterpolated(n) 
model_rev = models_list_fwd_trigram[1] #KneserNeyInterpolated(n)
model.fit(train_data, padded_sents)
print(model.vocab)
len(model.vocab)

model_rev.fit(train_data_rev, padded_sents_rev)
print(model_rev.vocab)
len(model_rev.vocab)

print(model.vocab.lookup(tokenized_text[0]))
print(model.counts)

#'standardized_text' : "634 368 002 061 717 390"
    
print("count of 390:", model.counts['390'])
print(model.counts[['717']]['390'])
print(model.counts[['368', '002']]['061'])
print(model.counts[['002', '061']]['717'])
print(model.score('390'))
# lm.score("b", ["a"]) what is the chance that “b” is preceded by “a”.
print("---", model.score('390', ['717']))

print(model.score('390', '717'.split()))  # P('390'|'717) Given 415 occurs what is the prob of 390
print(model.score('061', '717'.split()))  # P('740'|'390)
print(model.score('368', '002 061'.split()))  # P('368|'002 061')
print(model.score('002', '0061 717'.split()))

print("Entropy and Perplexity")

test = [('634', '368'), ('002', '061')]
print(model.entropy(test))
print(model.perplexity(test))

<Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
<Vocabulary with cutoff=1 unk_label='<UNK>' and 101 items>
('705', '050', '740')
<NgramCounter with 3 ngram orders and 4023 ngrams>
count of 390: 30
2
2
2
0.023529411764705882
--- 0.15235294117647058
0.15235294117647058
0.1504705882352941
7.058823529411765e-05
0.15541176470588236
Entropy and Perplexity
3.1407477080968564
8.819810801552226


In [19]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize 

def generate_sent(model, num_words,char_seed, random_seed=42):
    """
    :param model: An ngram language model.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, text_seed=char_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>'  or token == '< /s>' :
            break
        content.append(token)
    return detokenize(content)

In [20]:
#print(generate_sent(model,1,['717','061'],42))
#print(generate_sent(model,1,['002','368'],42))

print(generate_sent(model,1,['072'],42))
print(generate_sent(model,1,['740','017','495'],42))
print(generate_sent(model,1,['017','495'],42))
print(generate_sent(model,1,['495'],42))
print(generate_sent(model,1,['741'],42))

#Text 113 and #114
#822 140 015 R/L
#375 400 390 632 R/L

print("-----------------")
print(generate_sent(model,1,['822','140'],42))
print(generate_sent(model,1,['140'],42))

print(generate_sent(model,1,['375', '400','390'],42))
print(generate_sent(model,1,['400','390'],42))
print(generate_sent(model,1,['390'],42))

#'634 368 002 061 717 390' Standardized text - L to R like in English

print("More Tests")
print(generate_sent(model,1,['634','368'],42))
print(generate_sent(model,1,['368','002'],42))
print(generate_sent(model,1,['002'],42))

print(generate_sent(model,1,['002','061'],42))
print(generate_sent(model,1,['061','717'],42))


</ s>
2 3 5
2 3 5
2 3 5
4 5 6
-----------------
7 4 0
7 4 0
</ s>
</ s>
</ s>
More Tests
2 9 8
2 2 0
8 1 7
7 1 7
3 9 0


In [21]:
def pack_list(first_param, second_param, third_param):
    packed_list = []
    
    if(third_param!=-1):
        packed_list.append(third_param)
        
    if(second_param!=-1):
        packed_list.append(second_param)
        
    if(first_param!=-1):
        packed_list.append(first_param)
  
    return packed_list

In [22]:
def get_list_token(terminal, j, list_tokens, index_unclear_signs, k,p):
    one_before=-1
    two_before=-1
    three_before=-1
    param =[]
    last_token_index = len(list_tokens)-1 
    #print(terminal, j, list_tokens, index_unclear_signs, k,p)
    
    try:
        if(terminal):
            if(index_unclear_signs[j]+k<=last_token_index):
                one_before = list_tokens[index_unclear_signs[j]+k]

            if(index_unclear_signs[j]+p<=last_token_index):
                two_before = list_tokens[index_unclear_signs[j]+p]
        else:
            if(index_unclear_signs[j]+k>=0):
                one_before= list_tokens[index_unclear_signs[j]+k]

            if(index_unclear_signs[j]+p>=0):
                two_before = list_tokens[index_unclear_signs[j]+p]


        param = pack_list(one_before, two_before, three_before)
    except Exception as e:
        print("Exception:", e.__class__, "occurred in get_list_token.")
    return param

In [23]:
def find_unclear_characters(model1, model2, df_Items, seed=8, forward=True):
    #For each of the text with unclear character go through it
    
    ans=-1
    unclear_chars = []
    for text in a :
        try:
            #Identify the position of the unclear text and get its immediate neighbors
            list_tokens = nltk.word_tokenize(text)
            last_token_index = len(list_tokens)-1  
            index_unclear_signs = []
            param =[]

            for i in range(0, last_token_index+1) :
                    if(list_tokens[i]=='000'):
                        index_unclear_signs.append(i)

            print("Text:Index of Unclear signs:", str(text) + ": " + str(index_unclear_signs))

            #assuming one unclear sign in a text. TBD: Extend this later to more than one unclear sign
            j=0
            if(index_unclear_signs[j]==0):
                # case of LtoR: Terminal char is unclear
                print("LtoR: Terminal char is unclear")
                param = get_list_token(True, j, list_tokens, index_unclear_signs, 1,2)
                print("Sending: ", param , " to generate next char")
                #Use reverse model
                ans = generate_sent(model2, 1, param , random_seed=seed)
                unclear_chars.append(ans)


            elif(index_unclear_signs[j]==last_token_index):
                # case of LtoR: Beginning char is unclear
                print("LtoR: Beginning char is unclear")
                param = get_list_token(False, j, list_tokens, index_unclear_signs, -1,-2)
                print("Sending: ", param , " to generate next char")
                # for this use the model_rev to generate the character
                ans = generate_sent(model1, 1, param, random_seed=seed)
                unclear_chars.append(ans)


            else:
                # case of LtoR: medial sign is unclear
                print("LtoR: One of the middle char is unclear")
                #Not proceeding if more than one char is unclear

                if(len(index_unclear_signs)>1):
                    print("Many chars are unclear in the text, not able to decipher the text. Moving on ...\n")
                    continue
                param = get_list_token(False,j, list_tokens, index_unclear_signs, -1,-2)
                print("Sending: ", param , " to generate next char")
                ans = generate_sent(model1, 1, param, random_seed=seed)
                unclear_chars.append(ans)
        except Exception as e:
            print("Exception:", e.__class__, "find_unclear_characters.")
            
            
    return  unclear_chars

## TESTING
1. Test 1: Use a sample of Training data but one character is made unclear: Take a few samples from Training data, make some characters as 000 (unclear). Do it in begining, terminal and medial regions of the text and see if the model is able to figure out the unclear characters
2. Test 2: Use Testing data: Take the Testing data and make some charcters as 000 (unclear). Do it in begining, terminal and medial regions of the text to test and see if the model is able to figure out the unclear characters
3. Test 3: Real Unclear texts: Use the actual texts with unclear data and what the model is able to come up with for the unclear characters

# Test 1


In [24]:
# Test1
# Data preparation for test
# Take three rows from general set, convert a known sign to unclear sign and produce a dataframe


#LtoR: Terminal unclear sign
made_up_from_train_row1 = {'site' : 'fake',
       'reversed_text' : '000 717 061 002 368 634',
       'standardized_text' : "634 368 002 061 717 000"}
df_made_up_from_train= pd.DataFrame([made_up_from_train_row1])


#LtoR: Medial unclear sign
made_up_from_train_row2= {'site' : 'fake',
       'reversed_text'  : '390 717 061 000 368 634',
       'standardized_text' : "634 368 000 061 717 390"}
df_made_up_from_train= df_made_up_from_train.append(made_up_from_train_row2 , 
                    ignore_index=True)

#LtoR: Beginning unclear sign
made_up_from_train_row3= {'site' : 'fake',
       'reversed_text'  : '390 717 061 002 368 000',
       'standardized_text' : "000 368 002 061 717 390"}
df_made_up_from_train= df_made_up_from_train.append(made_up_from_train_row3 , 
                    ignore_index=True)

test1_correct_answers = ['390','002','634']

# Check if 'standardized_text' : "634 368 002 061 717 390" was in train set
sample = df_train_x[df_train_x.standardized_text=='634 368 002 061 717 390'].standardized_text
print("Sample:\n", sample)

Sample:
 8    634 368 002 061 717 390
Name: standardized_text, dtype: object


/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_61904/2536741739.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_made_up_from_train= df_made_up_from_train.append(made_up_from_train_row2 ,
/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_61904/2536741739.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_made_up_from_train= df_made_up_from_train.append(made_up_from_train_row3 ,


In [25]:
def check_answers(ans, test_correct_answers):
    hit=0
    i=0
    for answers in ans:
        if(answers.replace(" ", "")==test_correct_answers[i]):
            hit=hit+1
        i=i+1
    return hit
    

In [26]:


# Try unclear texts in fwd direction
print("_____________________________")
print("_____ Running TEST-1_________")
print("_____________________________")

print("_____Trying unclear texts in forward direction:_______")

try:
    fwd_trigram_model_hit = [0,0,0,0,0]
    a = df_made_up_from_train[df_made_up_from_train.standardized_text!=''].standardized_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_fwd_trigram[index], models_list_rev_trigram[index],a, 8,True)
        fwd_trigram_model_hit[index] = check_answers(ans,test1_correct_answers)
        print("Fwd_Trigram_model:", model_name_list[index], " identified", fwd_trigram_model_hit[index], "out of", len(test1_correct_answers), " Answers:", ans)
except Exception as e:
            print("Exception:", e.__class__)


# Try unclear strings in reverse
print("_____Trying unclear texts in reverse:______")
try:
    rev_trigram_model_hit = [0,0,0,0,0]
    a= df_made_up_from_train[df_made_up_from_train.reversed_text!=''].reversed_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_rev_trigram[index], models_list_rev_trigram[index],a, 8,True)
        rev_trigram_model_hit[index] = check_answers(ans,test1_correct_answers)
        print("Rev_Trigram_model:", model_name_list[index], " identified", rev_trigram_model_hit[index], "out of", len(test1_correct_answers), " Answers:", ans)
except Exception as e:
            print("Exception:", e.__class__)


_____________________________
_____ Running TEST-1_________
_____________________________
_____Trying unclear texts in forward direction:_______
***********Model Name*****************: MLE
Text:Index of Unclear signs: 634 368 002 061 717 000: [5]
LtoR: Beginning char is unclear
Sending:  ['061', '717']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 634 368 000 061 717 390: [2]
LtoR: One of the middle char is unclear
Sending:  ['634', '368']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 000 368 002 061 717 390: [0]
LtoR: Terminal char is unclear
Sending:  ['002', '368']  to generate next char
Fwd_Trigram_model: MLE  identified 0 out of 3  Answers: ['5 5 0']
***********Model Name*****************: KneserNeyInterpolated
Text:Index of Unclear signs: 634 368 002 061 717 000: [5]
LtoR: Beginning char is unclear
Sending:  ['061', '717']  to generate next char
Text:Index

# Test 2

In [27]:
def reverse_text(a):
    list_reversed_text = []
    for text in a :
        # Tokenize to words
        # first split the string into chars
        chars = text.split(' ')

        # then reverse the split string list and join with a space
        reversed_text = ' '.join(reversed(chars))
        list_reversed_text.append(reversed_text)
    return list_reversed_text 
    

In [28]:
# Test 2
# Data preparation for test
# Take rows from Testing set, convert a known sign to unclear sign and produce a dataframe and feed it

import random

list_changed_texts = []
list_changed_reversed_text = []
test2_correct_answers= []

for text in df_test_x[df_test_x.standardized_text!=''].standardized_text:
    # Tokenize to words, first split the string into chars
    chars = text.split(' ')
    new_text = chars
    #randomly pick an index in the tokenized_text and change it to unclear
    r = random.randrange(0, len(chars)-1)
    test2_correct_answers.append(chars[r])
    
    new_text[r]= '000'
    # then join with a space
    changed_text = ' '.join((new_text))
    list_changed_texts.append(changed_text)
    
    
df_test_x['changed_text']= list_changed_texts
list_changed_reversed_text = reverse_text(df_test_x[df_test_x.changed_text!=''].changed_text)
df_test_x['changed_reversed_text']= list_changed_reversed_text


print(df_test_x)


             standardized_text                 changed_text  \
0          861 002 032 176 740          861 000 032 176 740   
1      817 002 032 220 231 740      000 002 032 220 231 740   
2                      803 740                      000 740   
3                      061 740                      000 740   
4  413 125 350 297 002 003 226  413 125 350 297 002 000 226   
5              033 125 390 368              033 125 000 368   
6                 055[ 740 790                  000 740 790   

         changed_reversed_text  
0          740 176 032 000 861  
1      740 231 220 032 002 000  
2                      740 000  
3                      740 000  
4  226 000 002 297 350 125 413  
5              368 000 125 033  
6                  790 740 000  


In [29]:

print("_____________________________")
print("_____ Running TEST-2_________")
print("_____________________________")

# Test text made intentionally unclear
# Try unclear texts in fwd direction
print("_____Trying unclear texts in forward direction:_______")

try:
    fwd_trigram_model_hit = [0,0,0,0,0]
    a = df_test_x[df_test_x.changed_text!=''].changed_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_fwd_trigram[index], models_list_rev_trigram[index],a, 8,True)
        fwd_trigram_model_hit[index] = check_answers(ans, test2_correct_answers)
        print("Fwd_Trigram_model:", model_name_list[index], " identified", fwd_trigram_model_hit[index], "out of", len(test2_correct_answers), " Answers:", ans)
except Exception as e:
    print("Exception:", e.__class__)


 # Try unclear strings in reverse
print("_____Trying unclear texts in reverse:______")
    
try:
    rev_trigram_model_hit = [0,0,0,0,0]
    a= df_test_x[df_test_x.changed_reversed_text!=''].changed_reversed_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_rev_trigram[index], models_list_rev_trigram[index],a, 8,True)
        rev_trigram_model_hit[index] = check_answers(ans,test2_correct_answers)
        print("Rev_Trigram_model:", model_name_list[index], " identified", rev_trigram_model_hit[index], "out of", len(test2_correct_answers), " Answers:", ans)

except Exception as e:
    print("Exception:", e.__class__)

_____________________________
_____ Running TEST-2_________
_____________________________
_____Trying unclear texts in forward direction:_______
***********Model Name*****************: MLE
Text:Index of Unclear signs: 861 000 032 176 740: [1]
LtoR: One of the middle char is unclear
Sending:  ['861']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 000 002 032 220 231 740: [0]
LtoR: Terminal char is unclear
Sending:  ['032', '002']  to generate next char
Text:Index of Unclear signs: 000 740: [0]
LtoR: Terminal char is unclear
Sending:  ['740']  to generate next char
Text:Index of Unclear signs: 000 740: [0]
LtoR: Terminal char is unclear
Sending:  ['740']  to generate next char
Text:Index of Unclear signs: 413 125 350 297 002 000 226: [5]
LtoR: One of the middle char is unclear
Sending:  ['297', '002']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 033 125 000 368: [

# Test 3

In [30]:
# Actual Unclear texts
# Try unclear texts in fwd direction

print("_____________________________")
print("_____ Running TEST-3_________")
print("_____________________________")

print("_____Trying unclear texts in forward direction:_______")

try:
    fwd_trigram_model_hit = [0,0,0,0,0]
    a = df_unclear[df_unclear.standardized_text!=''].standardized_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_fwd_trigram[index], models_list_rev_trigram[index],a, 8,True)
        print("Fwd_Trigram_model:", model_name_list[index], " identified", "Answers:", ans)
except Exception as e:
    print("Exception:", e.__class__)


 # Try unclear strings in reverse
print("_____Trying unclear texts in reverse:______")
    
try:
    rev_trigram_model_hit = [0,0,0,0,0]
    a= df_unclear[df_unclear.reversed_text!=''].reversed_text
    for index in range(0, len(model_name_list)):
        print("***********Model Name*****************:", model_name_list[index])
        ans = find_unclear_characters(models_list_rev_trigram[index], models_list_rev_trigram[index],a, 8,True)
        print("Rev_Trigram_model:", model_name_list[index], " identified", "Answers:", ans)

except Exception as e:
    print("Exception:", e.__class__)

_____________________________
_____ Running TEST-3_________
_____________________________
_____Trying unclear texts in forward direction:_______
***********Model Name*****************: MLE
Text:Index of Unclear signs: 000 220: [0]
LtoR: Terminal char is unclear
Sending:  ['220']  to generate next char
Text:Index of Unclear signs: 000 388 000 000 142 740: [0, 2, 3]
LtoR: Terminal char is unclear
Sending:  ['000', '388']  to generate next char
Text:Index of Unclear signs: 000 861 002 503 440 740: [0]
LtoR: Terminal char is unclear
Sending:  ['002', '861']  to generate next char
Text:Index of Unclear signs: 455 004 000: [2]
LtoR: Beginning char is unclear
Sending:  ['455', '004']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 415 000: [1]
LtoR: Beginning char is unclear
Sending:  ['415']  to generate next char
Exception: <class 'ValueError'> find_unclear_characters.
Text:Index of Unclear signs: 000 000 233 388 000: [0, 1, 4]
Lt

Text:Index of Unclear signs: 000 002 861: [0]
LtoR: Terminal char is unclear
Sending:  ['861', '002']  to generate next char
Text:Index of Unclear signs: 906 090 000 158 900 380 484 900: [2]
LtoR: One of the middle char is unclear
Sending:  ['906', '090']  to generate next char
Text:Index of Unclear signs: 000 000 000 031 700: [0, 1, 2]
LtoR: Terminal char is unclear
Sending:  ['000', '000']  to generate next char
Text:Index of Unclear signs: 740 000[: [1]
LtoR: One of the middle char is unclear
Sending:  ['740']  to generate next char
Text:Index of Unclear signs: 700 000: [1]
LtoR: Beginning char is unclear
Sending:  ['700']  to generate next char
Text:Index of Unclear signs: 000 000 000: [0, 1, 2]
LtoR: Terminal char is unclear
Sending:  ['000', '000']  to generate next char
Text:Index of Unclear signs: 220 000 740 550 705: [1]
LtoR: One of the middle char is unclear
Sending:  ['220']  to generate next char
Text:Index of Unclear signs: 000 000 492[: [0, 1]
LtoR: Terminal char is uncl

# Other Tests

In [31]:
#Test:
print(model.generate(5, random_seed=7))
print(model_rev.generate(5, random_seed=7))

print(model.generate(2,text_seed=['390'], random_seed=7))
print(generate_sent(model, 2, ['390'], random_seed=8))
print(generate_sent(model, 2, ['390', '717'], random_seed=8))
print(generate_sent(model_rev, 2, ['634', '368'], random_seed=8))
print(generate_sent(model_rev, 2, ['861'], random_seed=8))
print(generate_sent(model_rev, 2, ['861', '002'], random_seed=8))
print(generate_sent(model, 1, ['390', '717', '061'], random_seed=8))

['226', '032', '817', '</s>', '</s>']
['226', '032', '817', '</s>', '</s>']
['590', '031']
161
061 002
002 220
002 840
125
0 0 2


In [32]:
# LTR Terminal Character is unclear
generate_sent(model_rev, 1, ['717'], random_seed=8)

'0 6 1'

In [33]:
# Medial Character is unclear
generate_sent(model, 1, ['717', '061'], random_seed=8)

'0 0 2'

In [34]:
# LTR Beg Character is unclear
generate_sent(model, 1, ['002', '368'], random_seed=8)

'5 5 0'